# Tool Calling

In [1]:
import dotenv
from agents import Agent, ModelSettings, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [25]:
@function_tool
def get_food_calories(food_item: str) -> str:
    """
    Get calorie information for common foods to help with nutrition tracking.

    Args:
        food_item: Name of the food (e.g., "apple", "banana")

    Returns:
        Calorie information per standard serving
    """
    # Simple calorie database - in real world, you'd use USDA API
    calorie_data_dict = {
        "apple": "80 calories per medium apple (182g)",
        "banana": "105 calories per medium banana (118g)",
        "broccoli": "25 calories per 1 cup chopped (91g)",
        "almonds": "164 calories per 1oz (28g) or about 23 nuts",
        "pear": "100 calories per medium pear (178g)",
        "peanuts": "161 calories per 1oz (28g)",
        "chicken breast": "165 calories per 3oz (85g) cooked",
        "rice": "206 calories per 1 cup cooked (158g)",
        "salmon": "233 calories per 3oz (85g) cooked",
        "egg": "78 calories per large egg (50g)"
    }

    food_key = food_item.lower()
    if food_key in calorie_data_dict:
        return f"{food_item.title()}: {calorie_data_dict[food_key]}"
    else:
        return f"I don't have calorie data for {food_item} in my database. Try common foods like apple, chicken breast, or rice."

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `get_food_calories` function_

In [7]:
get_food_calories('banana')

'Banana: 105 calories per medium banana (118g)'

In [28]:
calorie_agent = Agent(
    name="Gav's Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories]
    )

In [29]:
with trace("Many calls to Function agent for egg and pear"):
    result = await Runner.run(
        calorie_agent, "How many calories are in total in an human and a pear?"
    )
    print(result.final_output)

I can’t help with calories for a human. For a pear: about 100 calories per medium pear (178 g). If you want a total with another food, tell me the item and size.


Enforce tools use:

In [ ]:
calorie_agent2 = Agent(
    name="Gav's Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories],
    model_settings=ModelSettings(tool_choice="get_food_calories"),
)

with trace("Nutrition Assistant Run8 with forced tool choice"):
    result = await Runner.run(
        calorie_agent2, "How many calories are in total in a peanut and an pear?"
    )
    print(result.final_output)

About 200 calories total (medium banana ~105 + medium apple ~95). Variations: small banana ~90, large ~135; apples ~95 (medium).
